In [0]:
# All imports needed
import pyspark.sql.functions as F
from pyspark.sql.window import Window

# Just for fun
from pyspark.testing import assertDataFrameEqual

In [0]:
# Variables / parameters
dbutils.widgets.text("bucket_name", "af-rearc-quest", "AWS S3 Bucket Name")
bucket_name = dbutils.widgets.get("bucket_name")

In [0]:
# Helper Functions
def read_from_s3(path: str, format: str, **options):
    reader = spark.read.format(format)

    if options:
        reader = reader.options(**options)

    return reader.load(path)

# Just for fun
def dataframe_equal(df1, df2):
    try:
        assertDataFrameEqual(df1, df2)
        return ("DataFrames are equal.\n")
    except AssertionError as e:
        return (f"DataFrames are not equal. Differences:\n{e}\n")

In [0]:
# Read S3 Files into Data Frames and display them
## Read BLS/pr.data.0.Current into Dataframe
bls_df = read_from_s3(path="s3://af-rearc-quest/BLS/pr.data.0.Current",
                           format="csv",
                           sep="\t",
                           header=True,
                           ignoreLeadingWhiteSpace=True,
                           ignoreTrailingWhiteSpace=True,
                           inferSchema=True)
display(bls_df)

## Read honolulu-api/yearly_population.json into Dataframe
api_df = read_from_s3(path="s3://af-rearc-quest/honolulu-api/yearly_population.json",
                      format="json")
display(api_df)

series_id,year,period,value,footnote_codes
PRS30006011,1995,Q01,2.6,null
PRS30006011,1995,Q02,2.1,null
PRS30006011,1995,Q03,0.9,null
PRS30006011,1995,Q04,0.1,null
PRS30006011,1995,Q05,1.4,null
PRS30006011,1996,Q01,-0.2,null
PRS30006011,1996,Q02,-0.3,null
PRS30006011,1996,Q03,-0.1,null
PRS30006011,1996,Q04,0.2,null
PRS30006011,1996,Q05,-0.1,null


annotations,columns,data,page
"List(http://www.census.gov/programs-surveys/acs/, ACS 1-year Estimate, The American Community Survey (ACS) is conducted by the US Census and sent to a portion of the population every year., Census Bureau, Demographics, B01003, Diversity)","List(Nation ID, Nation, Year, Population)","List(List(United States, 01000US, 3.16128839E8, 2013), List(United States, 01000US, 3.18857056E8, 2014), List(United States, 01000US, 3.21418821E8, 2015), List(United States, 01000US, 3.23127515E8, 2016), List(United States, 01000US, 3.25719178E8, 2017), List(United States, 01000US, 3.27167439E8, 2018), List(United States, 01000US, 3.28239523E8, 2019), List(United States, 01000US, 3.31893745E8, 2021), List(United States, 01000US, 3.33287562E8, 2022), List(United States, 01000US, 3.34914896E8, 2023))","List(0, 0, 10)"


In [0]:
# Transform honolulu-api dataframe to retrieve the data column as multiple rows
api_df = api_df.select("data").withColumn("data", F.explode(F.col("data")))
display(api_df)

# Transform honolulu-api dataframe to retrieve the data column as a proper table
api_df = api_df.withColumn("Nation", F.col("data.Nation"))\
               .withColumn("Nation_ID", F.col("data.`Nation ID`"))\
               .withColumn("Population", F.col("data.Population"))\
               .withColumn("Year", F.col("data.Year"))\
               .select("Nation", "Nation_ID", "Population", "Year")
display(api_df)

data
"List(United States, 01000US, 3.16128839E8, 2013)"
"List(United States, 01000US, 3.18857056E8, 2014)"
"List(United States, 01000US, 3.21418821E8, 2015)"
"List(United States, 01000US, 3.23127515E8, 2016)"
"List(United States, 01000US, 3.25719178E8, 2017)"
"List(United States, 01000US, 3.27167439E8, 2018)"
"List(United States, 01000US, 3.28239523E8, 2019)"
"List(United States, 01000US, 3.31893745E8, 2021)"
"List(United States, 01000US, 3.33287562E8, 2022)"
"List(United States, 01000US, 3.34914896E8, 2023)"


Nation,Nation_ID,Population,Year
United States,01000US,3.16128839E8,2013
United States,01000US,3.18857056E8,2014
United States,01000US,3.21418821E8,2015
United States,01000US,3.23127515E8,2016
United States,01000US,3.25719178E8,2017
United States,01000US,3.27167439E8,2018
United States,01000US,3.28239523E8,2019
United States,01000US,3.31893745E8,2021
United States,01000US,3.33287562E8,2022
United States,01000US,3.34914896E8,2023


In [0]:
# Creates Dataframe for mean and the standard deviation of the annual US population across the years [2013, 2018] inclusive.
api_pop_calcs_df = api_df.where((api_df.Year >= 2013) & (api_df.Year <= 2018))\
                         .agg(F.mean("Population").alias("Mean_Population"), F.stddev("Population").alias("Standard_Dev_Population"))\
                         .select("Mean_Population", "Standard_Dev_Population")
display(api_pop_calcs_df)

Mean_Population,Standard_Dev_Population
3.22069808E8,4158441.040908095


In [0]:
# Create windowSpec for window function usage
windowSpec = Window.partitionBy("series_id").orderBy(F.desc("value"))

In [0]:
# Creates the Dataframe for the best year by series and displays it
bls_best_year_by_series_df = bls_df.groupBy("series_id","year")\
                                   .agg(F.sum("value").alias("value"))\
                                   .withColumn("row_num", F.row_number().over(windowSpec))\
                                   .where(F.col("row_num") == 1)\
                                   .select("series_id","year","value")
display(bls_best_year_by_series_df)

series_id,year,value
PRS30006011,2022,20.5
PRS30006012,2022,17.1
PRS30006013,1998,705.895
PRS30006021,2010,17.7
PRS30006022,2010,12.399999999999999
PRS30006023,2014,503.21600000000007
PRS30006031,2022,20.5
PRS30006032,2021,17.1
PRS30006033,1998,702.672
PRS30006061,2022,34.5


In [0]:
bls_api_df = bls_df.join(api_df, bls_df.year == api_df.Year, "left")\
                   .where((bls_df.series_id == 'PRS30006032') & (bls_df.period == 'Q01'))\
                   .select("series_id",bls_df.year,"period","value","Population")
display(bls_api_df)

series_id,year,period,value,Population
PRS30006032,1995,Q01,0.0,null
PRS30006032,1996,Q01,-4.2,null
PRS30006032,1997,Q01,2.8,null
PRS30006032,1998,Q01,0.9,null
PRS30006032,1999,Q01,-4.1,null
PRS30006032,2000,Q01,0.5,null
PRS30006032,2001,Q01,-6.3,null
PRS30006032,2002,Q01,-6.6,null
PRS30006032,2003,Q01,-5.7,null
PRS30006032,2004,Q01,2.0,null


In [0]:
# Data Analytics
## BLS/pr.data.0.Current - Best Year by Series: 
### bls_best_year_by_series_df

## honolulu-api/yearly_population.json - Value and Population by Year for series_id = PRS30006032 and period = Q01:
### bls_api_df